# Dependences

In [ ]:
include("../../src/struct_data.jl")

# Model

In [ ]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

# Aggregate

In [ ]:
agg = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time agg = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    model
)
# show_aggregates(agg)

# kNN

In [ ]:
include("../../src/neighbor.jl")
println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

# Force

## Function

In [ ]:
# N_i = zeros(1,size(agg.Position,1)) |> cu

In [ ]:
include("../../src/forces/forces.jl")

threads=(64,3)
@cuda(
    threads = threads,
    blocks = cld.(size(agg.Position,),threads),
    shmem=prod(threads.+2)*sizeof(Float32),
    sum_force!(
        agg.Position,
        agg.Simulation.Force.F,
        agg.Simulation.Force.Pol,
        agg.Simulation.Force.N_i,
        agg.Simulation.Neighbor.idx_sum,
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Parameter.Force,
        agg.Simulation.Parameter.Contractile.fₚ,
        atan(1),
        pi/4,
        model.Time.dt
    )
)

# sum(agg.Simulation.Force.Pol.^2, dims=2)
# # display(sum(.!(0.0 .< agg.Simulation.Force.Pol .< 1.0),dims=1))

# # display(agg.Simulation.Force.F)
# # display(agg.Position)
# # display(agg.Position)

open("Pol.xyz", "w") do f
    write(f, "$(size(agg.Simulation.Force.Pol, 1))\n")
    write(f, "t=0\n")
    writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Simulation.Force.Pol)), ' ')
end

open("Force.xyz", "w") do f
    write(f, "$(size(agg.Simulation.Force.F, 1))\n")
    write(f, "t=0\n")
    writedlm(f,hcat(agg.Geometry.outline,Matrix(agg.Simulation.Force.F)), ' ')
end
# agg.Simulation.Force.F